# Выбор локации для скважины

### Цель
 Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль на основании информации о трех регионах нефтедобычи. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error as mse

In [ ]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
print(df0.info())
print()
print(df1.info())
print()
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column

Видно, что датафреймы содержат по 10000 объектов. Все признаки объектов количественные.

In [ ]:
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


По условию, значения в столбце id уникальны, значит они дублируют функцию индекса датафрейма. Удалим их.

In [ ]:
df0, df1, df2 = df0.drop('id', axis = 1), df1.drop('id', axis = 1), df2.drop('id', axis = 1)

Разделим выборки на тестовые и валидационные

In [ ]:
def split(data):
    features = data.drop('product', axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = 123456)
    return features_train, features_valid, target_train, target_valid

Для удобства заключим тестовые и валидационные выборки каждого датасета в ассоциативный массив

In [ ]:

samples = {}
df_list = [df0, df1, df2]
for i,j in zip(df_list, range(3)):
    temp_list = []
    features_train, features_valid, target_train, target_valid = split(i)

    temp_list.append(features_train)
    temp_list.append(features_valid)
    temp_list.append(target_train)
    temp_list.append(target_valid)

    samples[j] = temp_list


Стандартизируем данные:

In [ ]:
def standartize(features, scaler):
    arr = scaler.transform(features)
    features_st = pd.DataFrame(arr, columns = features.columns)
    return features_st

for i in samples:
    scaler = StandardScaler()
    scaler.fit(samples[i][0])
    samples[i][0] = standartize(samples[i][0], scaler)
    samples[i][1] = standartize(samples[i][1], scaler)

Данные готовы для обучения модели

## Обучение и проверка модели

In [ ]:
predictions_list = []

for i in range(3):
    model = LinearRegression()
    model.fit(samples[i][0], samples[i][2])
    predictions = model.predict(samples[i][1])
    predictions = pd.Series(predictions, index = samples[i][3].index)
    predictions_list.append(predictions)
    product_mean = predictions.mean()
    rmse = mse(samples[i][3], predictions, squared = False)

    print('region',i,'product_mean:',product_mean,'|', 'rmse:', rmse)


region 0 product_mean: 92.49286560032228 | rmse: 37.80046993478272
region 1 product_mean: 69.03055429593262 | rmse: 0.8904933206270059
region 2 product_mean: 94.84835708988288 | rmse: 39.98039278339241


В результате обучения трех моделей для предсказания количества сырья для каждого региона стало ясно, что наибольшее среднее количество сырья приходится на нулевой и второй регион. Однако наибольшей точности достигла модель, обучающаяся на датасете первого региона: RMSE здесь составила 0.89

## Подготовка к расчёту прибыли

Рассчитаем необходимый средний минимум добычи в регионе:

In [ ]:
barell_profit = 450_000
budget = 10**10
min_amount = budget/(barell_profit*200)
print(min_amount)

111.11111111111111


Необходимый минимум добычи из скважины оказался больше, чем средние значения что предсказала каждая из моделей. Пока что можно предположить, что наименее перспективным для инвестиций будет регион номер один, так как метрика RMSE модели, предсказывающей количество продукта для данного региона оказалась очень близка к нулю, а само среднее значение - ниже необходимого минимума. Однако, в выборке мог быть большой разброс данных, и так как необходимо выбрать регион лишь по 200 лучшим скважинам, то выводы стоит делать уже после проведения бутстрепа.

Напишем функцию для определения прибыли по выбранным скважинам и предсказаниям модели

In [ ]:
def get_profit(target, predicted):
    best_preds = predicted.sort_values(ascending = False)
    best_wells = target[best_preds.index][:200]
    income = best_wells.sum() * barell_profit
    return income - budget

## Расчёт прибыли и рисков

Проведем бутстреп:

In [ ]:
state = np.random.RandomState(12345)

for i in range(3):
    profit_list = []
    for j in range(1000):
        subsample = samples[i][3].sample(n = 500, replace=True, random_state=state)
        predictions = predictions_list[i][subsample.index]
        profit = get_profit(subsample, predictions)
        profit_list.append(profit)

    profit_list = pd.Series(profit_list)
    lower = profit_list.quantile(0.025)
    upper = profit_list.quantile(0.975)
    risk = (profit_list < 0).mean()
    print('---------------Регион', i, ":--------------------")
    print('Доверительный интервал: от', lower, 'до',upper)
    print('Риск уйти в минус:', risk)
    print()

---------------Регион 0 :--------------------
Доверительный интервал: от -44127667.247095145 до 1018572401.7182902
Риск уйти в минус: 0.034

---------------Регион 1 :--------------------
Доверительный интервал: от 90961941.11778933 до 918656675.8367736
Риск уйти в минус: 0.01

---------------Регион 2 :--------------------
Доверительный интервал: от -112083846.38237537 до 931287435.5839065
Риск уйти в минус: 0.067



По результатам бутстрепа теперь можно с уверенностью сказать, что не стоит сторой регион рассматривать - слишком большой риск уйти в минус, и нижний порог доверительного интервала в 95 процентов отрицателен. Теперь также известно, что не лучшей идеей будет инвестировать в развитие нулевого региона: хоть верхний порог доверительного интервала и самый высокий из всех регионов, нижни порог отрицательный, поэтому тут так же есть риск получить отрицательную прибыль. Самый оптимальный вариант - регион номер 1.